In [36]:
import pandas as pd

pollution = pd.read_csv('./data/pollution_us_2000_2016.csv.tar.gz')
pollution = pollution.drop(pollution.columns[0], axis=1)

In [39]:
# --- Missing values analysis ---

# Get all rows that contain at least one NaN
missings = pollution[pollution.isna().any(axis=1)]

# How many observations with missings are there?
print("Number of rows with missings: " + str(len(missings)))
print("Number of rows in the dataset: " + str(len(pollution)))

# Get missings count for all axes
# Apparently only SO2 AQI and CO AQI include missing values
# We shall see if we can impute those later
# As we can already observe, the missings of those 2 columns seem
# to originate mostly from different observations
axis_counts = missings.count(axis = 0).rsub(len(missings))
axis_counts

Number of rows with missings: 1309785
Number of rows in the dataset: 1746661


State Code                0
County Code               0
Site Num                  0
Address                   0
State                     0
County                    0
City                      0
Date Local                0
NO2 Units                 0
NO2 Mean                  0
NO2 1st Max Value         0
NO2 1st Max Hour          0
NO2 AQI                   0
O3 Units                  0
O3 Mean                   0
O3 1st Max Value          0
O3 1st Max Hour           0
O3 AQI                    0
SO2 Units                 0
SO2 Mean                  0
SO2 1st Max Value         0
SO2 1st Max Hour          0
SO2 AQI              872907
CO Units                  0
CO Mean                   0
CO 1st Max Value          0
CO 1st Max Hour           0
CO AQI               873323
dtype: int64